In [1]:
import numpy as np
import nltk
import os
import ast
import pickle
import tensorflow as tf
import random
import configparser
from tqdm import tqdm
import string

In [2]:
cwd = os.getcwd()
#print(cwd)
corpusDir = os.path.join(cwd, 'data/cornell')
#print(corpusDir)

In [3]:
lines = {}
with open(os.path.join(corpusDir, 'movie_lines.txt'), 'r', encoding='iso-8859-1') as f:
    for line in f:
        #print(line)
        fields = line.split(' +++$+++ ')
        #print(fields)
        obj = {}
        obj['lineID'] = fields[0]
        obj['characterID'] = fields[1]
        obj['movieID'] = fields[2]
        obj['characterName'] = fields[3]
        obj['text'] = fields[4]
        lines[fields[0]] = obj
#print(lines)

In [4]:
conversations = []
with open(os.path.join(corpusDir, 'movie_conversations.txt'), 'r', encoding='iso-8859-1') as f:
    for line in f:
        #print(line)
        fields = line.split(' +++$+++ ')
        #print(fields)
        obj = {}
        obj['character1ID'] = fields[0]
        obj['character2ID'] = fields[1]
        obj['movieID'] = fields[2]
        #obj['lineIDs'] = fields[3]
        #print(obj)
        lineIDs = ast.literal_eval(fields[3])
        #print(lineIDs)
        obj['lineIDs'] = lineIDs
        #print(obj)
        obj['lines'] = []
        for lineID in lineIDs:
            #print(lineID, "--", lines[lineID])
            obj['lines'].append(lines[lineID])
        conversations.append(obj)
#print(conversations)

In [5]:
wordIDMap = {}
IDWordMap = {}
unknownToken = -1
trainingSamples = []
goToken = -1
eosToken = -1
padToken = -1
sentMaxLength = 10 #maximum length of an input or output sentence
encoderMaxLength = sentMaxLength
decoderMaxLength = sentMaxLength + 2

In [6]:
def getWordID(word, shouldAddToDict=True):
    word = word.lower()
    wordID = wordIDMap.get(word, -1)
    if wordID == -1:
        if shouldAddToDict:
            wordID = len(wordIDMap)
            wordIDMap[word] = wordID
            IDWordMap[wordID] = word
        else:
            wordID = unknownToken
    return wordID

In [7]:
def getWordsFromLine(line, isReply=False):
    '''Returns the word IDs from the vovabulary'''
    words = []
    sentences = nltk.sent_tokenize(line)
    #print(sentences)
    # Since we are limited by a maxmimum length of sentences, we keep the last lines if the statement is a question/input
    # and the first few lines if the statement is an answer/reply
    for i in range(len(sentences)):
        if not isReply:
            i = len(sentences) - 1 - i
        tokensFromCurrSent = nltk.word_tokenize(sentences[i])
        #print(tokensFromCurrSent)
        if len(words) + len(tokensFromCurrSent) > sentMaxLength:
            break
        else:
            temp = []
            for token in tokensFromCurrSent:
                temp.append(getWordID(token))
            if isReply:
                words = words + temp
            else:
                words = temp + words # Append in the reverse order because we're considering the last few lines
    return words

In [8]:
padToken = getWordID('<pad>')
unknownToken = getWordID('<unknown>')
eosToken = getWordID('<eos>')
goToken = getWordID('<go>')
for conversation in conversations:
    #print(conversation)
    for i in range(len(conversation['lines']) - 1):
        #print(conversation['lines'][i])
        inputStatement = conversation['lines'][i]
        #print(inputStatement)
        replyStatement = conversation['lines'][i + 1]
        inputWords = getWordsFromLine(inputStatement['text'])
        replyWords = getWordsFromLine(replyStatement['text'], True)
        #print(inputWords)
        #print(replyWords)
        
        if inputWords and replyWords:
            trainingSamples.append([inputWords, replyWords])
#print(trainingSamples)

In [9]:
print("Saving dataset samples ...")
with open(os.path.join(cwd, 'data/samples', 'sampleData.pkl'), 'wb') as f:
    data = {
        'wordIDMap': wordIDMap,
        'IDWordMap': IDWordMap,
        'trainingSamples': trainingSamples
    }
    pickle.dump(data, f, -1)
print('Done')

Saving dataset samples ...
Done


In [33]:
#Parameters
globalStep = 0
cellUnitCount = 512
numOfLayers = 2
embeddingSize = 64
learningRate = 0.02
batchSize = 256
dropout = 0.9
softmaxSamples = 0
numOfEpochs = 2

In [34]:
def make_lstm_cell():
    encoderDecoderCell = tf.contrib.rnn.BasicLSTMCell(cellUnitCount)
    encoderDecoderCell = tf.contrib.rnn.DropoutWrapper(encoderDecoderCell, input_keep_prob=1.0, output_keep_prob=dropout)
    return encoderDecoderCell

In [35]:
#Expand the list comprehension below
encoderDecoderCell = tf.contrib.rnn.MultiRNNCell(
    [make_lstm_cell() for _ in range(numOfLayers)],
)

In [36]:
with tf.name_scope('encoder'):
    encoderInputs = [tf.placeholder(tf.int32, [None, ]) for _ in range(sentMaxLength)]
with tf.name_scope('decoder'):
    decoderInputs = [tf.placeholder(tf.int32, [None, ], name="inputs") for _ in range(sentMaxLength + 2)]
    decoderTargets = [tf.placeholder(tf.int32, [None, ], name="targets") for _ in range(sentMaxLength + 2)]
    decoderWeights = [tf.placeholder(tf.float32, [None, ], name="weights") for _ in range(sentMaxLength + 2)]

In [37]:
#Verify this - is different from the existing
decoderOutput, state = tf.contrib.legacy_seq2seq.embedding_rnn_seq2seq(
    encoderInputs,
    decoderInputs,
    encoderDecoderCell,
    len(wordIDMap),
    len(wordIDMap),
    embeddingSize,
    output_projection=None,
    feed_previous=True
)

ValueError: Variable embedding_rnn_seq2seq/rnn/embedding_wrapper/embedding already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "/home/sayak/tensorflow/lib/python3.5/site-packages/tensorflow/contrib/rnn/python/ops/core_rnn_cell_impl.py", line 593, in __call__
    dtype=data_type)
  File "/home/sayak/tensorflow/lib/python3.5/site-packages/tensorflow/contrib/rnn/python/ops/core_rnn.py", line 184, in <lambda>
    call_cell = lambda: cell(input_, state)
  File "/home/sayak/tensorflow/lib/python3.5/site-packages/tensorflow/contrib/rnn/python/ops/core_rnn.py", line 197, in static_rnn
    (output, state) = call_cell()


In [38]:
lossFunc = tf.contrib.legacy_seq2seq.sequence_loss(
    decoderOutput,
    decoderTargets,
    decoderWeights,
    len(wordIDMap),
    softmax_loss_function=None
)
tf.summary.scalar('loss', lossFunc)

<tf.Tensor 'loss_1:0' shape=() dtype=string>

In [39]:
optimizer = tf.train.AdamOptimizer(
    learning_rate=learningRate,
    beta1=0.9,
    beta2=0.999,
    epsilon=1e-08
)
optimizationOperation = optimizer.minimize(lossFunc)

ValueError: Variable embedding_rnn_seq2seq/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/weights/Adam/ already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-16-48bb81c77adb>", line 7, in <module>
    optimizationOperation = optimizer.minimize(lossFunc)
  File "/home/sayak/tensorflow/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/home/sayak/tensorflow/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2821, in run_ast_nodes
    if self.run_code(code, result):


In [40]:
writer = tf.summary.FileWriter('seq2seq')
saver = tf.train.Saver(max_to_keep=200)

In [41]:
sess = tf.Session(
    config=tf.ConfigProto(
        allow_soft_placement=True,
        log_device_placement=False
    )
)
sess.run(tf.global_variables_initializer())

In [42]:
#Change variable scope name
with tf.variable_scope("embedding_rnn_seq2seq/rnn/embedding_wrapper", reuse=True):
    in_embedding = tf.get_variable("embedding")
with tf.variable_scope("embedding_rnn_seq2seq/embedding_rnn_decoder", reuse=True):
    out_embedding = tf.get_variable("embedding")

embedding_vars = tf.get_collection_ref(tf.GraphKeys.TRAINABLE_VARIABLES)
embedding_vars.remove(in_embedding)
embedding_vars.remove(out_embedding)

'''
if globalStep != 0:
    return
'''

ValueError: list.remove(x): x not in list

In [43]:
with open(os.path.join(cwd, 'data/word2vec/GoogleNews-vectors-negative300.bin'), "rb", 0) as f:
    header = f.readline().split()
    #print(header)
    vocabulary_size = int(header[0])
    word_vector_size = int(header[1])
    #print('{}, {}'.format(vocabulary_size, word_vector_size))
    binary_length = np.dtype('float32').itemsize * word_vector_size
    #print(binary_length)
    initial_weights = np.random.uniform(-0.25, 0.25, (len(wordIDMap), word_vector_size))
    #print(initial_weights)
    for line in range(word_vector_size):
        word = []
        while True:
            ch = f.read(1)
            if ch == b' ':
                word = b''.join(word).decode('utf-8')
                break
            if ch != b'\n':
                word.append(ch)
        if word in wordIDMap:
            initial_weights[wordIDMap[word]] = np.fromstring(f.read(binary_length), dtype='float32')
        else:
            f.read(binary_length)

In [44]:
if embeddingSize < word_vector_size:
    u, s, vt = np.linalg.svd(initial_weights, full_matrices=False)
    S = np.zeros((word_vector_size, word_vector_size), dtype=complex)
    S[:word_vector_size, :word_vector_size] = np.diag(s)
    initial_weights = np.dot(u[:, :embeddingSize], S[:embeddingSize, :embeddingSize])

In [45]:
sess.run(in_embedding.assign(initial_weights))
sess.run(out_embedding.assign(initial_weights))

/home/sayak/tensorflow/lib/python3.5/site-packages/tensorflow/python/framework/tensor_util.py:355: ComplexWarning: Casting complex values to real discards the imaginary part
  nparray = values.astype(dtype.as_numpy_dtype)


array([[ 0.02442564,  0.01449201,  0.17819051, ...,  0.29747248,
        -0.22108015, -0.19583753],
       [ 0.09369779,  0.07363001,  0.07704376, ..., -0.18864214,
         0.19162565, -0.10645648],
       [-0.10043826, -0.03329198,  0.20276904, ...,  0.03705681,
         0.22164914, -0.07328618],
       ..., 
       [-0.03504413,  0.16652584, -0.20387016, ..., -0.0523993 ,
        -0.12524308,  0.06757042],
       [ 0.1423067 , -0.22834739, -0.01715017, ..., -0.13829713,
        -0.10723506, -0.20472412],
       [ 0.19271727, -0.13890515,  0.12240385, ..., -0.11258908,
         0.24868603, -0.18852122]], dtype=float32)

In [46]:
def generateNextSample():
    for i in range(0, len(trainingSamples), batchSize):
        yield trainingSamples[i:min(i + batchSize, len(trainingSamples))]

In [47]:
class Batch:
    def __init__(self):
        self.encoderSeqs = []
        self.decoderSeqs = []
        self.targetSeqs = []
        self.weights = []

In [48]:
def saveModel():
    print('Saving model checkpoint...')
    model_name = 'model_' + str(globalStep) + '.ckpt'
    if globalStep == 30:
        model_name = 'model.ckpt'
    saver.save(sess, os.path.join(cwd, 'saved_model', model_name))
    print('Done')

In [49]:
def makeBatch(samples):
    batch = Batch()
    batchSize = len(samples)
    for i in range(batchSize):
        sample = samples[i]
        #print(sample)
        batch.encoderSeqs.append(list(reversed(sample[0])))
        batch.decoderSeqs.append([goToken] + sample[1] + [eosToken])
        batch.targetSeqs.append(batch.decoderSeqs[-1][1:])

        batch.encoderSeqs[i] = [padToken] * (encoderMaxLength - len(batch.encoderSeqs[i])) + batch.encoderSeqs[i]
        batch.weights.append([1.0] * len(batch.targetSeqs[i]) + [0.0] * (decoderMaxLength - len(batch.targetSeqs[i])))
        batch.decoderSeqs[i] = batch.decoderSeqs[i] + [padToken] * (decoderMaxLength - len(batch.decoderSeqs[i]))
        batch.targetSeqs[i] = batch.targetSeqs[i] + [padToken] * (decoderMaxLength - len(batch.targetSeqs[i]))

    encoderSeqListT = []
    for i in range(encoderMaxLength):
        encoderSeqT = []
        for j in range(batchSize):
            encoderSeqT.append(batch.encoderSeqs[j][i])
        encoderSeqListT.append(encoderSeqT)
    batch.encoderSeqs = encoderSeqListT

    decoderSeqListT = []
    targetSeqListT = []
    weightListT = []
    for i in range(decoderMaxLength):
        decoderSeqT = []
        targetSeqT = []
        weightT = []
        for j in range(batchSize):
            #print('j: {}, i:{}'.format(j,i))
            decoderSeqT.append(batch.decoderSeqs[j][i])
            targetSeqT.append(batch.targetSeqs[j][i])
            weightT.append(batch.weights[j][i])
        decoderSeqListT.append(decoderSeqT)
        targetSeqListT.append(targetSeqT)
        weightListT.append(weightT)
    batch.decoderSeqs = decoderSeqListT
    batch.targetSeqs = targetSeqListT
    batch.weights = weightListT
    return batch

In [51]:
# Training Loop
completeSummary = tf.summary.merge_all()
if globalStep == 0:
    writer.add_graph(sess.graph)
try:
    for epoch in range(numOfEpochs):
        print("\nEpoch {}".format(epoch+1))
        random.shuffle(trainingSamples)
        
        batches = []
        for samples in generateNextSample():
            #print(samples)
            batch = makeBatch(samples)
            batches.append(batch)
        
        for batch in tqdm(batches, desc="Training"):
            print(batch.encoderSeqs)
            feedDict = {}
            ops = None
            for i in range(encoderMaxLength):
                feedDict[encoderInputs[i]] = batch.encoderSeqs[i]
            for i in range(decoderMaxLength):
                feedDict[decoderInputs[i]] = batch.decoderSeqs[i]
                feedDict[decoderTargets[i]] = batch.targetSeqs[i]
                feedDict[decoderWeights[i]] = batch.weights[i]
            ops = (optimizationOperation, lossFunc)
            assert len(ops) == 2
            #print(feedDict)
            _, loss, summary = sess.run(ops + (completeSummary,), feedDict)
            writer.add_summary(summary, globalStep)
            globalStep += 1
            if globalStep % 100 == 0:
                perplexity = math.exp(float(loss))
                print("Step %d " % (globalStep))
                print("Loss %.2f" % (loss))
                print("Perplexity %.2f" % (perplexity))
            if globalStep % 10 == 0:
                saveModel()
except (KeyboardInterrupt, SystemExit):
    print('Exiting')
saveModel()
sess.close()


Epoch 1


Training:   0%|          | 0/547 [00:00<?, ?it/s]

[[0, 0, 0, 5, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 72, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 18, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 18, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 146, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 4946, 0, 0, 170, 5, 0, 0, 0, 0, 0, 36, 0, 0, 0, 0, 0, 0, 0, 0, 146, 0, 542, 18696, 0, 0, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 418, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1411

InvalidArgumentError: You must feed a value for placeholder tensor 'encoder/Placeholder' with dtype int32
	 [[Node: encoder/Placeholder = Placeholder[dtype=DT_INT32, shape=[], _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'encoder/Placeholder', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/sayak/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/sayak/tensorflow/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/sayak/tensorflow/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/home/sayak/tensorflow/lib/python3.5/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, s

In [28]:
saved_model_dir = 'saved_model'
model_name = 'model_30.ckpt'
if os.path.exists(os.path.join(cwd, saved_model_dir, model_name)):
    print('Restoring model {}'.format(model_name))
    saver.restore(sess, os.path.join(cwd, saved_model_dir, model_name))
    print('Welcome to DeepChat! I am Alex. You can ask me questions and ponder on the answers I provide.')
    print('Type \'exit\' to end the chat.')
    while True:
        user_input = input('You: ')
        if user_input == '':
            print('Alex: Please say something! I don\'t like silence!')
        if user_input == 'exit':
            break
        inputSequence = []
        tokens = nltk.word_tokenize(user_input)
        if len(tokens) > sentMaxLength:
            print('I didn\'t understand! Please try a smaller sentence')
            continue
        wordIDs = []
        for token in tokens:
            wordIDs.append(getWordID(token, shouldAddToDict=False))
        batch = makeBatch([[wordIDs,[]]])
        inputSequence.extend(batch.encoderSeqs)
        feedDict = {}
        ops = None
        for i in range(encoderMaxLength):
            feedDict[encoderInputs[i]] = batch.encoderSeqs[i]
        feedDict[decoderInputs[0]] = [goToken]
        #print('decoderOutput {}'.format(decoderOutput))
        ops = (decoderOutput,)
        outputs = sess.run(ops[0], feedDict)
        outputSequence = []
        for output in outputs:
            outputSequence.append(np.argmax(output))
        #print('outputSequence {}'.format(outputSequence))
        responseTokens = []
        for wordID in outputSequence:
            if wordID == eosToken:
                break
            elif wordID != padToken and wordID != goToken:
                responseTokens.append(IDWordMap[wordID])
        #print('responseTokens {}'.format(responseTokens))
        response = ""
        responseTokens = [t.replace(t, ' ' + t) if not t.startswith('\'') and t not in string.punctuation else t for t in responseTokens]
        #print('responseTokens after replace {}'.format(responseTokens))
        response = ''.join(responseTokens).strip().capitalize()
        print('Alex: ' + response)
        print()

Restoring model model_30.ckpt
Welcome to DeepChat! I am Alex. You can ask me questions and ponder on the answers I provide.
Type 'exit' to end the chat.
You: hello
Alex: I

You: i what?
Alex: I

You: exit
